In [7]:
import boto3
import json
import random
bedrock_runtime=boto3.client("bedrock-runtime",region_name="us-east-1")
bedrock = boto3.client(service_name="bedrock",region_name="us-east-1")
s3 = boto3.client("s3")

model_id = "amazon.nova-reel-v1:0"
prompt ="A person posting as game character"

seed = random.randint(0,423982328)

model_input = {
    "taskType": "TEXT_VIDEO",
    "textToVideoParams": {"text": prompt},
    "videoGenerationConfig": {
        "fps": 24,
        "durationSeconds": 6,
        "dimension": "1280x720",
        "seed": seed,
    },
}

output_config = {
    "s3OutputDataConfig": {
        "s3Uri": "s3://gen-ai-exercise-shivam/video/"
    }
}

response = bedrock_runtime.start_async_invoke(
    modelId=model_id,
    modelInput=model_input,
    outputDataConfig=output_config,
)

invocation_arn = response["invocationArn"]
print("✅ Job submitted!")
print("Invocation ARN:", invocation_arn)



✅ Job submitted!
Invocation ARN: arn:aws:bedrock:us-east-1:637626381203:async-invoke/69w2qb293mxw


In [10]:
job_status = bedrock_runtime.get_async_invoke(invocationArn=invocation_arn)
print("Current Status:", job_status["status"])

Current Status: Completed


In [12]:
#Text Generation

In [13]:
import boto3
import json

MODEL_ID = "amazon.nova-micro-v1:0"
bedrock = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

response = bedrock.invoke_model(
    modelId=MODEL_ID,
    guardrailVersion="DRAFT",
    body=json.dumps({
        "schemaVersion": "messages-v1",
        "messages": [{"role": "user", "content": [{"text": "Rewrite this sentence for me in a formal tone: You are very good at your job."}]}],
        "inferenceConfig": {"maxTokens": 500, "topK": 20, "temperature": 0.7}
    })
)

response['body'].read()

b'{"output":{"message":{"content":[{"text":"You exhibit exceptional proficiency in your professional responsibilities."}],"role":"assistant"}},"stopReason":"end_turn","usage":{"inputTokens":19,"outputTokens":10,"totalTokens":29,"cacheReadInputTokenCount":0,"cacheWriteInputTokenCount":0}}'

In [15]:
# Using Amazon Bedrock for streaming text generation

In [16]:
import boto3
import json

bedrock = boto3.client('bedrock-runtime', region_name='us-east-1')
model_id = 'amazon.nova-micro-v1:0'

payload = {
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "text": "Tell me what type of dances people do."
                }
            ]
        }
    ]
}

payload_json = json.dumps(payload)

response = bedrock.invoke_model_with_response_stream(
    modelId=model_id,
    body=payload_json,
    contentType='application/json',
    accept='application/json'
)

stream = response['body']
full_text = ""

print("Receiving response stream:")
for event in stream:
    chunk = event['chunk']
    chunk_str = chunk.get('bytes', b'').decode('utf-8')

    try:
        json_chunk = json.loads(chunk_str)

        if "contentBlockDelta" in json_chunk:
            delta = json_chunk["contentBlockDelta"]["delta"]
            text = delta.get("text", "")
            full_text += text
            print(text, end='') 

    except json.JSONDecodeError:
        continue

Receiving response stream:
There are numerous types of dances that people perform around the world, each with its own unique style, history, and cultural significance. Here are some of the most popular and widely recognized types of dances:

### Traditional and Folk Dances
1. **Ballet** - A classical dance form that originated in the Italian Renaissance courts of the 15th century and later developed into a sophisticated, highly disciplined art form in France and Russia.
2. **Hip-Hop** - A cultural movement that includes dancing, rapping, DJing, and graffiti art, originating in the African-American and Latino communities in the United States.
3. **Salsa** - A lively and romantic dance that originated in the Caribbean and Latin America, characterized by intricate footwork and close partner interaction.
4. **Tango** - An elegant and passionate dance that originated in the working-class neighborhoods of Buenos Aires, Argentina, and Montevideo, Uruguay.
5. **Flamenco** - A passionate and ex